### Imports

In [25]:
import os
import json
import requests
from bs4 import BeautifulSoup

1. Extraction of opinions

In [18]:
with open("./cookie.json", "r", encoding="UTF-8") as jf:
   headers = json.load(jf)

In [30]:
product_id = "167976636"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while url:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            single_opinion = {}
            single_opinion['opinion_id'] = opinion["data-entry-id"]
            single_opinion['author'] = opinion.select_one("span.user-post__author-name").text.strip()
            try:
                single_opinion['recomend'] = opinion.select_one("span.user-post__author-recomendation > em").text.strip()
            except AttributeError:
                single_opinion['recomend'] = None
            single_opinion['stars'] = opinion.select_one("span.user-post__score-count").text.strip()
            single_opinion['content'] = opinion.select_one("div.user-post__text").text.strip()
            single_opinion['pros'] = [tag.text for tag in opinion.select("div.review-feature__item.review-feature__item--positive")]
            single_opinion['cons'] = [tag.text for tag in opinion.select("div.review-feature__item.review-feature__item--negative")]
            single_opinion['up_votes'] = opinion.select_one("button.vote-yes")["data-total-vote"].strip()
            single_opinion['down_votes'] = opinion.select_one("button.vote-no")["data-total-vote"].strip()
            single_opinion['published'] = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip()
            try:
                single_opinion['purchased'] = opinion.select_one("span.user-post__published > span > time:nth-child(2)")["datetime"].strip()
            except TypeError:
                single_opinion['purchased'] = None
            all_opinions.append(single_opinion)
        try:
            url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None
print(all_opinions)


[{'opinion_id': '19535811', 'author': 'a...h', 'recomend': 'Polecam', 'stars': '5/5', 'content': 'Świetna suszarka, spełnia wszystkie moje oczekiwania. Szybko suszy długie włosy. Mała, lekka i poręczna. Testuję dodatkowe końcówki do stylizacji. Jest też cicha suszarka więc nie mam już wyrzutów sumienia że budzę rodzinę nad ranem.', 'pros': ['bardzo estetyczny wy', 'głośność pracy', 'mały gabaryt', 'moc', 'szybkość nagrzewania', 'świetne dodatki', 'wygląd'], 'cons': [], 'up_votes': '0', 'down_votes': '0', 'published': '2025-03-19 16:39:41', 'purchased': '2025-03-03 07:48:53'}, {'opinion_id': '19170406', 'author': 'i...a', 'recomend': 'Polecam', 'stars': '5/5', 'content': 'Suszarka spełnia swoją funkcję, jest bardzo poręczna, mała ale o dużej mocy. Końcówki świetnie wygładzają lub kręcą włosy - zależnie od ustawień. Bardzo polecam ten produkt. Szczególnie w takiej cenie!', 'pros': ['głośność pracy', 'szybkość nagrzewania', 'wygląd'], 'cons': [], 'up_votes': '0', 'down_votes': '0', 'publi

### Creation of database

In [26]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [27]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)